In [2]:
from random import random

import flappy_bird_gymnasium
from gymnasium import envs

print(envs.registry.keys())
import numpy as np
import gymnasium

# Configurare
env = gymnasium.make("FlappyBird-v0", render_mode="human", use_lidar=True,)

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'phys2d/CartPole-v0', 'phys2d/CartPole-v1', 'phys2d/Pendulum-v0', 'LunarLander-v3', 'LunarLanderContinuous-v3', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v3', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'tabular/Blackjack-v0', 'tabular/CliffWalking-v0', 'Reacher-v2', 'Reacher-v4', 'Reacher-v5', 'Pusher-v2', 'Pusher-v4', 'Pusher-v5', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedPendulum-v5', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'InvertedDoublePendulum-v5', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'HalfCheetah-v5', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Hopper-v5', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Swimmer-v5', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Walker2d-v5', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Ant-v5', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'Humanoid-v5

In [22]:

# Q-Learning
alpha = 0.1
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
num_episodes = 100


In [4]:
from torchvision import transforms
preprocess = transforms.Compose([
    transforms.ToPILImage(),              # Conversie la imagine PIL
    transforms.Grayscale(num_output_channels=1),               # Convertire în tonuri de gri
    transforms.Resize((84,84)),         # Redimensionare la 84x84 pixeli
    transforms.ToTensor() # Conversie la tensor PyTorch
])

In [5]:
from collections import deque
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)  # Buffer cu lungime maximă
    
    def add(self, experience):
        self.buffer.append(experience)  # Adaugă experiența (state, action, reward, next_state, done)
    
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)  # Selectează un eșantion aleatoriu
    
    def __len__(self):
        return len(self.buffer)


In [9]:


replay_buffer = deque(maxlen=20000)  # Buffer pentru stocarea tranzițiilor


In [10]:
print(env.action_space)

Discrete(2)


In [64]:
import gym
import keyboard  # Pentru a detecta apăsările de taste


import numpy as np
from collections import deque

# Configurare environment
env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array" , use_lidar=False)

for i in range(100):
    
    # Joc manual
    state = env.reset()
    done = False
    count_frames=0

    # Inițializează primul frame procesat
    frame = env.render()
    processed_current_frame = preprocess(frame)     
    while not done:
        
        # Joacă manual (space = jump, altfel = stay)
        # if keyboard.is_pressed("space"):
        #     action = 1  # Jump
        # else:
        #     action = 0 
        # Stay
        if np.random.rand() < 0.1:
            action = 1
        else:
            action = 0 
    
        new_state, reward, done,_, info = env.step(action)
        
        next_frame = env.render()
        
        processed_next_frame = preprocess(next_frame)
        # Extrage array-ul de 12 valori din new_state
        """
        new_state contains :
        'last_pipe_horizontal_position',
        'last_top_pipe_vertical_position',
        'last_bottom_pipe_vertical_position'
        'next_pipe_horizontal_position'
        'next_top_pipe_vertical_position'
        'next_bottom_pipe_vertical_position',
        'next_next_pipe_horizontal_position'
        'next_next_top_pipe_vertical_position',
        'next_next_bottom_pipe_vertical_position',
        'player_vertical_position',
        'player_vertical_velocity',
        'player_rotation'
        """
        
        # Add to replay buffer
        if count_frames % 4 == 0 or done:
            replay_buffer.append((processed_current_frame, action, reward,processed_next_frame, new_state, done))
        
        # Actualize for next iteration
        processed_current_frame = processed_next_frame
        count_frames += 1
        if done:
            break
    

KeyboardInterrupt: 

In [65]:
print(f"Buffer size: {len(replay_buffer)}")
# print(f"buffer one example",replay_buffer[0],len(replay_buffer[0]))


Buffer size: 4855


In [ ]:
import numpy as np
from collections import deque

# Configurare environment
env = gymnasium.make("FlappyBird-v0", render_mode="human" , use_lidar=False)
for i in range(num_episodes):
    
    # Joc manual
    state = env.reset()
    done = False
    count_frames=0
    
    frame =env.render() 
    print(" frame " ,frame)
    
    while not done:
        
        # Joacă manual (space = jump, altfel = stay)
        if keyboard.is_pressed("space"):
            action = 1  # Jump
        else:
            action = 0 

        new_state, reward, done, _ , info = env.step(action)  
        print("shape",new_state.shape)
        print("new state ",new_state )
        
        # Randează jocul
        frame =env.render() 
        print(" frame " ,frame)
        
        if done:
            break
                

In [ ]:
from matplotlib import pyplot as plt
# Assuming `processed_frame` is the grayscale image tensor

if len(replay_buffer) > 0:
    first_img = replay_buffer[0][0].squeeze(0)
else:
    first_img = None
    
def show_grayscale_image(image_tensor):
    plt.imshow(image_tensor.numpy(), cmap='gray')
    plt.title('Grayscale Image')
    plt.axis('off')
    plt.show()
    plt.pause(0.1)  # Pause to allow the image to be displayed

In [ ]:
# Example usage
for experience in replay_buffer:
    img = experience[0].squeeze(0)  # Extract the grayscale image tensor
    show_grayscale_image(img)
    img = experience[3].squeeze(0)  # Extract the grayscale image tensor
    show_grayscale_image(img)

In [17]:
# CNN
from torch import nn
from torch import optim
from tqdm import tqdm
import torch

class CNN(nn.Module):
    """
    Define the layers of the convolutional neural network.

    Parameters:
    in_channels: int
        The number of channels in the input image. For MNIST, this is 1 (grayscale images).
    num_classes: int
        The number of classes we want to predict, in our case 12 .
    """
    def __init__(self, in_channels, num_classes=12):
        super().__init__()
        # First convolutional layer: 1 input channel, 8 output channels, 3x3 kernel, stride 1, padding 1
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=5, stride=1, padding=0)
        # Max pooling layer: 2x2 window, stride 2
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        # Second convolutional layerl: 8 input channels, 16 output channels, 3x3 kernel, stride 1 , padding1
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16, kernel_size=5, stride=1,padding=0)
        # Second convolutional layerl: 8 input channels, 16 output channels, 3x3 kernel, stride 1 , padding1
        self.conv3 = nn.Conv2d(in_channels=16,out_channels=32, kernel_size=5, stride=1,padding=0)
        # Fully connected layer: 16 * 7 * 7  input features (after two 2x2 poolings), state output features (num_classes) 
        self.fc1 = nn.Linear(32*7*7,num_classes)

    def forward(self, x):
        """
        Define the forward pass of the neural network.

        Parameters:
            x: torch.Tensor
                The input tensor.

        Returns:
            torch.Tensor
                The output tensor after passing through the network.
        """
        x = torch.relu(self.conv1(x))  # Apply first convolution and ReLU activation
        x = self.pool(x)               # Apply max pooling
        x = torch.relu(self.conv2(x))  # Apply second convolution and ReLU activation
        x = self.pool(x)               # Apply max pooling
        x = torch.relu(self.conv3(x))  # Apply third convolution and ReLU activation
        x = self.pool(x)               # Apply max pooling
        x = x.view(x.size(0), -1)      # Flatten the tensor
        x = self.fc1(x)                # Apply fully connected layer
        return x
    

In [18]:
input_size = 7056  # 84x84 pixels (not directly used in CNN)
num_classes = 12 # 12 characteristics from use_lidar = False
learning_rate = 0.001
batch_size = 64
num_epochs = 10  

In [20]:
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

def train_feature_extractor(model, replay_buffer, batch_size, num_epochs, learning_rate):
    """
    Antrenează rețeaua CNN pentru extragerea caracteristicilor.

    Parameters:
        model: nn.Module
            Modelul CNN pentru extragerea caracteristicilor.
        replay_buffer: deque
            Lista cu experiențe (frame-uri, acțiuni, recompense, next_frame-uri, state-uri și done flag).
        batch_size: int
            Dimensiunea batch-ului.
        num_epochs: int
            Numărul de epoci.
        learning_rate: float
            Rata de învățare.
    """
    # Pregătește datele pentru antrenare
    frames = [experience[3] for experience in replay_buffer]  # Frame-urile curente
    targets = [experience[4] for experience in replay_buffer]  # Target-ul (new_state)

    # Convert frames și targets în tensors
    frames = torch.tensor(np.array(frames), dtype=torch.float32)
    targets = torch.tensor(np.array(targets), dtype=torch.float32)

    # Creează un dataset și un dataloader cu shuffle pe episoade
    dataset = TensorDataset(frames, targets)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Inițializează pierderea și optimizer-ul
    criterion = nn.MSELoss()  # Mean Squared Error Loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Activează modul de antrenare
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)  # Pierdere față de ținte
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


In [19]:
import torch
# [bird_y, bird_velocity, pipe_x1, pipe_x2]
feature_dim = 12  # Dimensiunea vectorului de caracteristici
model_actiune_curenta = CNN(in_channels=1, num_classes=feature_dim)  # Creare model CNN


In [25]:
train_feature_extractor(model_actiune_curenta, replay_buffer, batch_size, 50, learning_rate)  # Antrenare model


Epoch [1/50], Loss: 1.2567
Epoch [2/50], Loss: 1.1026
Epoch [3/50], Loss: 1.0455
Epoch [4/50], Loss: 0.9804
Epoch [5/50], Loss: 0.9094
Epoch [6/50], Loss: 0.8698
Epoch [7/50], Loss: 0.8004
Epoch [8/50], Loss: 0.7272
Epoch [9/50], Loss: 0.6817
Epoch [10/50], Loss: 0.6478
Epoch [11/50], Loss: 0.6069
Epoch [12/50], Loss: 0.5804
Epoch [13/50], Loss: 0.5425
Epoch [14/50], Loss: 0.5126
Epoch [15/50], Loss: 0.4963
Epoch [16/50], Loss: 0.4890
Epoch [17/50], Loss: 0.4557
Epoch [18/50], Loss: 0.4288
Epoch [19/50], Loss: 0.4259
Epoch [20/50], Loss: 0.4077
Epoch [21/50], Loss: 0.3981
Epoch [22/50], Loss: 0.3763
Epoch [23/50], Loss: 0.3694
Epoch [24/50], Loss: 0.3631
Epoch [25/50], Loss: 0.3529
Epoch [26/50], Loss: 0.3520
Epoch [27/50], Loss: 0.3370
Epoch [28/50], Loss: 0.3324
Epoch [29/50], Loss: 0.3355
Epoch [30/50], Loss: 0.3243
Epoch [31/50], Loss: 0.3161
Epoch [32/50], Loss: 0.3094
Epoch [33/50], Loss: 0.3073
Epoch [34/50], Loss: 0.3068
Epoch [35/50], Loss: 0.2959
Epoch [36/50], Loss: 0.2985
E

In [26]:
def validate_feature_extractor(model, validation_data, tolerance=0.1):
    """
    Evaluează modelul pe un set de validare și calculează MSE și acuratețea.

    Parameters:
        model: nn.Module
            Modelul CNN antrenat.
        validation_data: list of tuples
            Lista de (frame, target) pentru validare.
        tolerance: float
            Intervalul de toleranță pentru a considera o predicție corectă.

    Returns:
        mse: float
            Mean Squared Error pe setul de validare.
        accuracy: float
            Acuratețea în procente.
    """
    model.eval()  # Mod de evaluare (fără actualizare a ponderilor)
    mse_loss = nn.MSELoss()
    total_loss = 0
    correct_predictions = 0

    with torch.no_grad():  # Nu calculăm gradientele
        for frame, target in validation_data:
            frame = torch.tensor(frame, dtype=torch.float32).unsqueeze(0)  # Adaugă dimensiunea batch
            target = torch.tensor(target, dtype=torch.float32).unsqueeze(0)
            prediction = model(frame)

            loss = mse_loss(prediction, target)
            total_loss += loss.item()

            # Verificăm dacă predicția este în toleranță
            if torch.all(torch.abs(prediction - target) < tolerance):
                correct_predictions += 1

    mse = total_loss / len(validation_data)
    accuracy = (correct_predictions / len(validation_data)) * 100
    return mse, accuracy


In [27]:
validation_data = []

In [29]:
import gym
import keyboard  # Pentru a detecta apăsările de taste

# Extract validation data

import numpy as np
from collections import deque

# Configurare environment
env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array" , use_lidar=False)

for i in range(10):
    
    # Joc manual
    state = env.reset()
    done = False
    count_frames=0

    # Inițializează primul frame procesat
    frame = env.render()
        
    while not done:
        
        # Joacă manual (space = jump, altfel = stay)
        # if keyboard.is_pressed("space"):
        #     action = 1  # Jump
        # else:
        #     action = 0 
        # Stay
        if np.random.rand() < 0.1:
            action = 1
        else:
            action = 0 
    
        new_state, reward, done,_, info = env.step(action)
        
        next_frame = env.render()
        
        processed_next_frame = preprocess(next_frame)
        # Extrage array-ul de 12 valori din new_state
        """
        new_state contains :
        'last_pipe_horizontal_position',
        'last_top_pipe_vertical_position',
        'last_bottom_pipe_vertical_position'
        'next_pipe_horizontal_position'
        'next_top_pipe_vertical_position'
        'next_bottom_pipe_vertical_position',
        'next_next_pipe_horizontal_position'
        'next_next_top_pipe_vertical_position',
        'next_next_bottom_pipe_vertical_position',
        'player_vertical_position',
        'player_vertical_velocity',
        'player_rotation'
        """
        
        # Add to replay buffer
        if count_frames % 4 == 0 or done:
            validation_data.append((processed_next_frame,new_state))
        
        count_frames += 1
        if done:
            break

In [30]:
len(validation_data)

315

In [31]:
validate_feature_extractor(model_actiune_curenta, validation_data, tolerance=0.3)  # Validare model

C:\Users\Asus\AppData\Local\Temp\ipykernel_21800\1639196621.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  frame = torch.tensor(frame, dtype=torch.float32).unsqueeze(0)  # Adaugă dimensiunea batch


(0.006073929454982539, 79.04761904761905)

# Test Actor-Critic , nefunctional

Pas 1: Definirea Rețelelor Actor și Critic

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

# Actor Network
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim),
            nn.Softmax(dim=-1)  # Probabilitățile pentru acțiuni
        )

    def forward(self, state):
        return self.fc(state)

# Critic Network
class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Valoarea scalară a stării
        )

    def forward(self, state):
        return self.fc(state)


Pas 2: Inițializarea Mediului și Rețelelor

In [37]:
# Dimensiunea state-ului (12 caracteristici)
state_dim = 12
# Dimensiunea spațiului de acțiuni (2 acțiuni: flap sau no flap)
action_dim = 2

# Inițializăm Actorul și Criticul
actor = Actor(state_dim, action_dim)
critic = Critic(state_dim)

# Optimizatori
actor_optimizer = optim.Adam(actor.parameters(), lr=0.001)
critic_optimizer = optim.Adam(critic.parameters(), lr=0.001)

# Funcția de pierdere pentru Critic
critic_loss_fn = nn.MSELoss()


Pas 3: Funcții de Ajutor

In [55]:
#Alegerea unei acțiuni pe baza politicii:
def select_action(state, actor):
    state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convertim în tensor
    action_probs = actor(state_tensor)
    action = torch.multinomial(action_probs, 1).item()  # Alegem o acțiune
    
    return action, action_probs

def select_action_without_explore(state, actor):
    state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convertim în tensor
    action_probs = actor(state_tensor)
    action = torch.argmax(action_probs).item()  # Alegem acțiunea cu probabilitatea cea mai mare

    return action, action_probs


In [70]:
#Actualizarea Actorului:
def update_actor(actor, actor_optimizer, log_prob, advantage):
    advantage = torch.tensor(advantage, dtype=torch.float32)
    actor_loss = -(log_prob * advantage.detach()).mean()  # Gradientul politicii
    
    # Adăugăm regularizarea entropiei pentru explorare 
    # entropy_loss = -torch.sum(action_probs * torch.log(action_probs))
    # actor_loss += 0.01 * entropy_loss  # Adaugă regularizarea entropiei
    
    actor_optimizer.zero_grad()
    actor_loss.backward(retain_graph=True)
    actor_optimizer.step()


In [40]:
#Actualizarea Criticului:
def update_critic(critic, critic_optimizer, state, td_target,critic_loss_fn):
    state_tensor = torch.FloatTensor(state).unsqueeze(0)
    td_target_tensor = torch.FloatTensor([td_target])
    value = critic(state_tensor).squeeze(0)
    loss = critic_loss_fn(value, td_target_tensor)
    critic_optimizer.zero_grad()
    loss.backward()
    critic_optimizer.step()


Pas 4: Ciclu Principal de Învățare

In [ ]:
# vers 1
import numpy as np

# Hyperparametrii
gamma = 0.99  # Discount factor
num_episodes = 1000

for episode in range(num_episodes):
    state = env.reset()[0]  # Resetăm mediul (simulat sau real)
    done = False
    episode_reward = 0

    while not done:
        # 1. Alegem o acțiune
        action, action_probs = select_action(state, actor)
        log_prob = torch.log(action_probs[0, action])

        # 2. Executăm acțiunea și primim noua stare și recompensa
        next_state, reward, done, _, _ = env.step(action)
        episode_reward += reward 
        
        # 3. Calculăm valoarea TD Target și Advantage
        value = critic(torch.FloatTensor(state).unsqueeze(0)).item()
        next_value = critic(torch.FloatTensor(next_state).unsqueeze(0)).item() if not done else 0
        td_target = reward + gamma * next_value
        advantage = td_target - value

        # 4. Actualizăm Actorul și Criticul
        update_actor(actor, actor_optimizer, log_prob, advantage)
        update_critic(critic, critic_optimizer, state, td_target,critic_loss_fn)

        # 5. Trecem la următoarea stare
        state = next_state

    print(f"Episode {episode + 1}/{num_episodes}, Reward: {episode_reward}")


In [57]:
# vers 2
import numpy as np

# Hyperparametrii
gamma = 0.99  # Discount factor
epsilon = 0.3 # Valoarea inițială pentru explorare
epsilon_min = 0.1  # Valoarea minimă pentru explorare
epsilon_decay = 0.995  # Rata de scădere a epsilonului
num_episodes = 100

for episode in range(num_episodes):
    state = env.reset()[0]  # Resetăm mediul
    done = False
    episode_reward = 0

    while not done:
        # Explorare epsilon-greedy
        if np.random.rand() < epsilon:
            # Alegem o acțiune aleatorie pentru explorare
            action = np.random.choice([0, 1])
            log_prob = None  # Nu avem log_prob pentru acțiuni exploratorii
        else:
            # Alegem acțiunea pe baza actorului
            action, action_probs = select_action(state, actor)
            log_prob = torch.log(action_probs[0, action])

        # Executăm acțiunea în mediul de simulare
        next_state, reward, done, _ , _ = env.step(action)
        
        # Calculul centrului pipe-ului următor
        next_pipe_center = (state[4] + state[5]) / 2
        distance_from_center = abs(state[9] - next_pipe_center)

        # Penalizare pentru distanță față de centru
        reward -= distance_from_center * 0.01
        
        # Calculăm TD Target și Advantage
        value = critic(torch.FloatTensor(state).unsqueeze(0)).item()
        next_value = critic(torch.FloatTensor(next_state).unsqueeze(0)).item() if not done else 0
        td_target = reward + gamma * next_value
        advantage = td_target - value

        # Actualizăm Actorul doar dacă acțiunea este determinată de model
        if log_prob is not None:
            update_actor(actor, actor_optimizer, log_prob, advantage)

        # Actualizăm Criticul
        update_critic(critic, critic_optimizer, state, td_target,critic_loss_fn)

        # Actualizăm starea curentă
        state = next_state
        episode_reward += reward

    # Decay pentru epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    print(f"Episode {episode + 1}/{num_episodes}, Reward: {episode_reward}")


Episode 1/100, Reward: 6.618964843750001
Episode 2/100, Reward: 3.7777148437500006
Episode 3/100, Reward: 0.7657226562500001
Episode 4/100, Reward: -4.052578125
Episode 5/100, Reward: -0.4299023437500006
Episode 6/100, Reward: 3.8028124999999986
Episode 7/100, Reward: 3.81048828125
Episode 8/100, Reward: -1.65126953125
Episode 9/100, Reward: -3.4380859374999986
Episode 10/100, Reward: 1.3847265625
Episode 11/100, Reward: 3.84712890625
Episode 12/100, Reward: -0.43164062499999867
Episode 13/100, Reward: 3.8341015625
Episode 14/100, Reward: -1.0382031250000003
Episode 15/100, Reward: 3.7736132812500003
Episode 16/100, Reward: 3.8657031249999996
Episode 17/100, Reward: 3.7964453125
Episode 18/100, Reward: -3.4517968749999994
Episode 19/100, Reward: 3.8273437500000007
Episode 20/100, Reward: 3.8325390625000004
Episode 21/100, Reward: 0.7866406250000006
Episode 22/100, Reward: 3.8584570312500004
Episode 23/100, Reward: 3.7819531249999994
Episode 24/100, Reward: 3.805839843750001
Episode 25/

## Modelul actor-critic cu CNN


In [38]:
# Inițializăm Actorul și Criticul
actor2 = Actor(state_dim, action_dim)
critic2 = Critic(state_dim)

# Optimizatori
actor_optimizer2 = optim.Adam(actor2.parameters(), lr=0.001)
critic_optimizer2 = optim.Adam(critic2.parameters(), lr=0.001)

# Funcția de pierdere pentru Critic
critic_loss_fn2 = nn.MSELoss()

In [ ]:

import numpy as np

# Hyperparametrii
env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array" , use_lidar=False)

gamma = 0.99  # Discount factor
epsilon = 1.0  # Valoarea inițială pentru explorare
epsilon_min = 0.1  # Valoarea minimă pentru explorare
epsilon_decay = 0.995  # Rata de scădere a epsilonului
num_episodes = 1000
model_actiune_curenta.eval()

for episode in range(num_episodes):
    state = env.reset()[0]  # Resetăm mediul
    state_pixels = env.render()
    # print("state_pixels",state_pixels.shape)
    processed_frame = preprocess(state_pixels).unsqueeze(0)
    # print("processed_frame",processed_frame.shape)
    state = model_actiune_curenta(processed_frame)[0]
    state = state.detach().numpy()
    # print("state",state)
    # print("state",state)
    done = False
    episode_reward = 0

    while not done:
        # Explorare epsilon-greedy
        if np.random.rand() < epsilon:
            # Alegem o acțiune aleatorie pentru explorare
            action = np.random.choice([0, 1])
            log_prob = None  # Nu avem log_prob pentru acțiuni exploratorii
        else:
            # Alegem acțiunea pe baza actorului
            action, action_probs = select_action(state, actor2)
            log_prob = torch.log(action_probs[0, action])

        # Executăm acțiunea în mediul de simulare
        next_state_target , reward, done, _, _ = env.step(action)
        
        # Compute next state with our model CNN
        new_state_pixels = env.render()
        procesed_frame = preprocess(new_state_pixels).unsqueeze(0)
        next_state = model_actiune_curenta(procesed_frame)[0]
        next_state = next_state.detach().numpy()
        
        
        # print("next_state",next_state)
        # print("next_state_target",next_state_target)
        # Calculul centrului pipe-ului următor
        # next_pipe_center = (state[4] + state[5]) / 2
        # distance_from_center = abs(state[9] - next_pipe_center)
        # 
        # # Penalizare pentru distanță față de centru
        # reward -= distance_from_center * 0.01

        # Calculăm TD Target și Advantage
        value = critic2(torch.FloatTensor(state).unsqueeze(0)).item()
        next_value = critic2(torch.FloatTensor(next_state).unsqueeze(0)).item() if not done else 0
        td_target = reward + gamma * next_value
        advantage = td_target - value

        # Actualizăm Actorul doar dacă acțiunea este determinată de model
        if log_prob is not None:
            update_actor(actor2, actor_optimizer2, log_prob, advantage)

        # Actualizăm Criticul
        update_critic(critic2, critic_optimizer2, state, td_target,critic_loss_fn2)

        # Actualizăm starea curentă
        state = next_state
        episode_reward += reward
        # Add regularization noise for more robust exploration
        state = state + np.random.normal(0, 0.05, size=state.shape)  # Adaugă zgomot


    # Decay pentru epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    print(f"Episode {episode + 1}/{num_episodes}, Reward: {episode_reward}")


Episode 1/1000, Reward: -6.299999999999999
Episode 2/1000, Reward: -8.099999999999998
Episode 3/1000, Reward: -8.7
Episode 4/1000, Reward: -8.7
Episode 5/1000, Reward: -7.499999999999998
Episode 6/1000, Reward: -8.099999999999998
Episode 7/1000, Reward: -4.499999999999998
Episode 8/1000, Reward: -8.099999999999998
Episode 9/1000, Reward: -8.099999999999998
Episode 10/1000, Reward: -8.099999999999998
Episode 11/1000, Reward: -7.499999999999998
Episode 12/1000, Reward: -8.099999999999998
Episode 13/1000, Reward: -6.299999999999999
Episode 14/1000, Reward: -8.099999999999998
Episode 15/1000, Reward: -5.699999999999998
Episode 16/1000, Reward: -8.099999999999998
Episode 17/1000, Reward: -7.499999999999998
Episode 18/1000, Reward: -7.499999999999998
Episode 19/1000, Reward: -6.899999999999999
Episode 20/1000, Reward: -7.499999999999998
Episode 21/1000, Reward: -8.099999999999998
Episode 22/1000, Reward: -8.099999999999998
Episode 23/1000, Reward: -6.899999999999999
Episode 24/1000, Reward: 

Pas 5: Testare

In [71]:
env = gymnasium.make("FlappyBird-v0", render_mode="human" , use_lidar=False)
# print("Modelul Actor-Critic cu CNN",actor2)
# print("Modelul Actor-Critic fara CNN",actor)
for i in range(10):
    state = env.reset()[0]
    done = False
    while not done:
        action, _ = select_action_without_explore(state, actor2)
        # print("action",action)
        state, reward, done, _,_= env.step(action)
        env.render()  # Vizualizăm episodul


KeyboardInterrupt: 

# evaluare

In [ ]:
import matplotlib.pyplot as plt

def evaluate_model(env, actor, num_eval_episodes=100):
    scores = []
    survival_times = []
    pipes_passed = []

    for _ in range(num_eval_episodes):
        state = env.reset()[0]
        done = False
        episode_reward = 0
        frames = 0
        pipes = 0

        while not done:
            with torch.no_grad():
                action, _ = select_action_without_explore(state, actor)  # Politica deterministă (fără explorare)
            next_state, reward, done, info, _ = env.step(action)

            # Adunăm scorurile și statistici
            episode_reward += reward
            frames += 1
            if reward > 0.5:  # Ex: Recompensă mare pentru trecerea printr-un pipe
                pipes += 1

            state = next_state

        scores.append(episode_reward)
        survival_times.append(frames)
        pipes_passed.append(pipes)

    avg_score = np.mean(scores)
    avg_survival = np.mean(survival_times)
    avg_pipes = np.mean(pipes_passed)

    print(f"Evaluare pe {num_eval_episodes} episoade:")
    print(f"Scor mediu: {avg_score}")
    print(f"Timp mediu de supraviețuire: {avg_survival} cadre")
    print(f"Pipe-uri trecute în medie: {avg_pipes}")

    return scores, survival_times, pipes_passed

# După antrenare:
scores, survival_times, pipes_passed = evaluate_model(env, actor)

# Vizualizare
plt.figure(figsize=(12, 5))

# Recompensa medie
plt.subplot(1, 2, 1)
plt.plot(scores)
plt.title("Scor pe Episod")
plt.xlabel("Episod")
plt.ylabel("Scor")

# Supraviețuirea medie
plt.subplot(1, 2, 2)
plt.plot(survival_times)
plt.title("Timp de Supraviețuire pe Episod")
plt.xlabel("Episod")
plt.ylabel("Cadre")

plt.tight_layout()
plt.show()
